# Pré-processamento do ficheiro

Nesta sessão o objetivo é efetuar algum processamento usando dados de percursos registados por dispositivos GPS. Neste caso o percurso corresponde a uma volta de bicicleta. Os dados estão originalmente no formato GPX. Este formato é definido em XML e vamos começar pela sua trasnformação para um formato tabular, com um ponto do percurso por cada linha dum ficheiro CSV.

## Carregamento e interpretação do ficheiro GPX

Para nos ajudar a carregar o ficheiro GPX vamos usar a biblioteca xml. Esta biblioteca fornece-nos duas classes muito úteis:
- `ElementTree` - Classe que representa um ficheiro XML, com a sua hierarquia completa
- `Element` - Classe que representa um elemento da hierarquia (e os seus descendentes)

Para fazer a leitura do ficheiro GPX basta usar o método `parse(path)` de `xml.etree.ElementTree`. O nó que representa a raíz (um elemento) obtem-se usando o método `getroot()`:

In [1]:
import xml.etree.ElementTree as ET

tree = ET.parse('data/MAY_26_12_11_59_18.gpx')
root = tree.getroot()
root

<Element '{http://www.topografix.com/GPX/1/1}gpx' at 0x0000022C5C87C270>

Vamos de seguida obter todos os elementos que correspondem a pontos do percurso. Estes elementos estão identificados pela tag "trkpt" e podemos iterar sobre eles usando o método `iter(tagname)`. Neste exemplo estamos a construir uma lista definida por compreensão:

In [2]:
allPoints = [p for p in root.iter('{http://www.topografix.com/GPX/1/1}trkpt')]

Note-se que `iter(tagname)` não é uma lista, mas sim um iterador, que pode ser usado para iterarmos a sequência de elementos por ele definida. Podemos inspecionar o conteúdo das primeiras 5 posições do array, verificando que cada entrada é do tipo `Element`:

In [3]:
allPoints[:5]

[<Element '{http://www.topografix.com/GPX/1/1}trkpt' at 0x0000022C5C87C770>,
 <Element '{http://www.topografix.com/GPX/1/1}trkpt' at 0x0000022C5C87C900>,
 <Element '{http://www.topografix.com/GPX/1/1}trkpt' at 0x0000022C5C87CA40>,
 <Element '{http://www.topografix.com/GPX/1/1}trkpt' at 0x0000022C5C87CB80>,
 <Element '{http://www.topografix.com/GPX/1/1}trkpt' at 0x0000022C5C87CCC0>]

Os atributos de um `Element` podem ser obtidos usando o método `attrib()`, o qual devolve um dicionário. No caso dos elementos `trkpt`, os seus atributos são a latitude e longitude do ponto correspondente:

In [4]:
allPoints[0].attrib

{'lat': '38.6661240', 'lon': '-9.1681400'}

Cada `trkpt` pode possuir ainda dois subelementos, um para guardar a elevação e outro para guardar o instante (timestamp).

In [5]:
print('Elevação: ', allPoints[0].find('{http://www.topografix.com/GPX/1/1}ele').text)
print('Timestamp:', allPoints[0].find('{http://www.topografix.com/GPX/1/1}time').text)

Elevação:  20.0
Timestamp: 2012-05-26T08:23:50Z


In [6]:
def createGPXPointFromElement(elem):
    lat = float(elem.get('lat'))
    lon = float(elem.get('lon'))
    time = elem.find('{http://www.topografix.com/GPX/1/1}time').text
    ele = float(elem.find('{http://www.topografix.com/GPX/1/1}ele').text)
    return { 'lat': lat, 'lon': lon, 'ele': ele, 'time': time}

In [7]:
createGPXPointFromElement(allPoints[0])

{'lat': 38.666124,
 'lon': -9.16814,
 'ele': 20.0,
 'time': '2012-05-26T08:23:50Z'}

In [8]:
allGPXPoints = [createGPXPointFromElement(p) for p in allPoints]

In [9]:
allGPXPoints[:3]

[{'lat': 38.666124,
  'lon': -9.16814,
  'ele': 20.0,
  'time': '2012-05-26T08:23:50Z'},
 {'lat': 38.666117,
  'lon': -9.168181,
  'ele': 19.9,
  'time': '2012-05-26T08:23:51Z'},
 {'lat': 38.666094,
  'lon': -9.168354,
  'ele': 20.1,
  'time': '2012-05-26T08:23:55Z'}]

In [10]:
allGPXPoints[0]['time']

'2012-05-26T08:23:50Z'

Let's add a new 'place' field to each point and randomly...


In [11]:
for p in allGPXPoints:
    p['place'] = ''
    
allGPXPoints[0]['place'] = 'Start'
allGPXPoints[-1]['place'] = 'Finish' 

import random

nmarkers = 10
width = int(len(allGPXPoints) // nmarkers)

for i in range(1,nmarkers):
    r = i*width-random.randint(0,width//(nmarkers//4))
    allGPXPoints[r]['place'] = 'Marker#{}'.format(i)


In [12]:
# Open the file for writing
with open('data/MAY_26_12_11_59_18.csv', "w") as f:

    # Write the headers
    f.write("lat,lon,ele,time,place\n")

    for p in allGPXPoints:
        f.write('{:.7f},{:.7f},{:.1f},{},{}\n'.format(p['lat'], p['lon'], p['ele'], p['time'], p['place']))

## Notas:

- O processamento do ficheiro GPX poderia ter sido simplificado recorrendo à biblioteca [gpxpy](https://pypi.org/project/gpxpy/). Ao não usar essa biblioteca aproveitou-se para mostrar como se pode processar um ficheiro XML.